In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
from mprompt.modules.emb_diff_module import EmbDiffModule
import numpy as np
import matplotlib
import imodelsx.util
from copy import deepcopy
import re
import mprompt.viz
import scipy.special
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from mprompt.methods.m4_evaluate import D5_Validator
import openai
from mprompt.modules.fmri_module import fMRIModule

openai.api_key_path = os.path.expanduser('~/.OPENAI_KEY')


r = (pd.read_pickle('../results/results_fmri.pkl')
    .sort_values(by=['top_score_synthetic'], ascending=False))
r['id'] = "('" + r['top_explanation_init_strs'].str.replace(' ', '_').str.slice(stop=20) + "', '" + r['subject'] + "', " + r['module_num'].astype(str) + ")"

### Select voxels

In [56]:
# expls = ['baseball','animals','water','movement','religion','time','technology']
# interesting_expls = ['food', 'numbers', 'physical contact', 'time', 'laughter', 'age', 'clothing']
# voxels = [('movement', 'UTS01',	7), ('numbers', 'UTS03', 55), ('time', 'UTS03', 19), ('relationships', 'UTS01', 21),
        #   ('sounds', 'UTS03', 35), ('emotion', 'UTS03', 23), ('food', 'UTS03', 46)]
# voxels = [('numbers', 'UTS03', 55), ('time', 'UTS03', 19),
        #   ('sounds', 'UTS03', 35), ('emotion', 'UTS03', 23), ('food', 'UTS03', 46)]
# voxels = [('movement', 'UTS01',	7),('relationships', 'UTS01', 21) ('passing of time	UTS02	4)]
voxels = [('relationships', 'UTS02', 9), ('time', 'UTS02', 4), ('looking or staring', 'UTS03', 57)]

In [59]:
# convert voxels -> expl (and make sure it's corect)
rows = []
expls = []
for vox in voxels:
    expl, subj, vox_num = vox
    try:
        rows.append(r[(r.subject == subj) & (r.module_num == vox_num)].iloc[0])
        expls.append(expl)
    except:
        print('skipping', vox)
rows_check = pd.DataFrame(rows)
rows_check['expl'] = expls

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 200):
    display(rows_check[['subject', 'module_num', 'expl', 'top_explanation_init_strs', 'top_ngrams_module_correct']])

,subject,module_num,expl,top_explanation_init_strs,top_ngrams_module_correct
62,UTS02,9,relationships,relationships and milestones in life,"[boyfriend of six, dating for months, boyfriend a year, married for fifteen, we got engaged, we were engaged, twenty he retired, a divorce twenty, virginity at twenty, am turning forty, daughter i..."
39,UTS02,4,time,passing of time,"[weeks became months, weekends became weeks, five years four, the moment passed, twenty minutes thirty, more time passed, replied age thirty, moment passed, later came the, days later diagnosed, m..."
28,UTS03,57,looking or staring,looking or staring in some way,"[eyed her suspiciously, at him incredulously, wink at, at me shyly, locks eyes with, staring at me, turned and saw, and mimed crying, incredulously like look, staring right at, point at a, leered ..."


In [60]:
# display all rows
with pd.option_context('display.max_rows', None, 'display.max_colwidth', 200):
    display(r.sort_values(by=['top_score_synthetic'], ascending=False)[
        ['top_explanation_init_strs', 'subject', 'module_num', 'top_score_synthetic', 'frac_top_ngrams_module_correct', 'id', 'top_ngrams_module_correct']
    ].round(3).reset_index(drop=True).head(50))


,top_explanation_init_strs,subject,module_num,top_score_synthetic,frac_top_ngrams_module_correct,id,top_ngrams_module_correct
0,"birthdays, birth years, and zodiac signs",UTS03,39,0.150,0.187,"('birthdays,_birth_yea', 'UTS03', 39)","[identical birthday, this identical birthday, even my birthday, two identical birthday, piscean, on my birthday, piscean if, birth year so, birthday of, to my birthday, his birth year, birthday so..."
1,looking or staring in some way,UTS03,57,0.099,0.400,"('looking_or_staring_i', 'UTS03', 57)","[eyed her suspiciously, at him incredulously, wink at, at me shyly, locks eyes with, staring at me, turned and saw, and mimed crying, incredulously like look, staring right at, point at a, leered ..."
2,a specific year or decade in the 20th century,UTS02,24,0.097,0.773,"('a_specific_year_or_d', 'UTS02', 24)","[in nineteen fifty, of nineteen fifty, in nineteen forty, of nineteen eighty, nineteen forty, nineteenth nineteen forty, of nineteen forty, to nineteen forty, 's nineteen forty, in nineteen sevent..."
3,measurements of distance or time,UTS02,22,0.085,0.693,"('measurements_of_dist', 'UTS02', 22)","[fifty feet, fifteen twenty feet, crawl fifty meters, living fifty feet, every eight feet, shot just centimeters, to fifty feet, seventy to twenty, gun two inches, meters fifty feet, long sixty fo..."
4,a specific year in the 20th century,UTS02,5,0.076,0.760,"('a_specific_year_in_t', 'UTS02', 5)","[of nineteen fifty, nineteen sixty, of nineteen eighty, of nineteen sixty, nineteen ninety, nineteen eighty, nineteen seventy, nineteen forty, of nineteen forty, of nineteen ninety, nineteen fifty..."
5,a specific year,UTS02,16,0.076,0.467,"('a_specific_year', 'UTS02', 16)","[in nineteen fifty, in nineteen thirty, until nineteen sixty, in nineteen sixty, to nineteen forty, in nineteen forty, of nineteen eighty, nineteen forty, since nineteen eighty, nineteen sixty, un..."
6,relationships and milestones in life,UTS02,9,0.073,0.547,"('relationships_and_mi', 'UTS02', 9)","[boyfriend of six, dating for months, boyfriend a year, married for fifteen, we got engaged, we were engaged, twenty he retired, a divorce twenty, virginity at twenty, am turning forty, daughter i..."
7,task or responsibility that needs to be completed,UTS02,17,0.071,0.493,"('task_or_responsibili', 'UTS02', 17)","[she cooked dinner, me take dictation, receive the faxed, i shovel driveways, needed to photocopy, i wrote speeches, writing his obituary, delivered the food, bring me food, 'm signing books, sign..."
8,passing of time,UTS02,4,0.070,0.280,"('passing_of_time', 'UTS02', 4)","[weeks became months, weekends became weeks, five years four, the moment passed, twenty minutes thirty, more time passed, replied age thirty, moment passed, later came the, days later diagnosed, m..."
9,hair and clothing styles,UTS03,24,0.068,0.707,"('hair_and_clothing_st', 'UTS03', 24)","[hair in pigtails, pigtails with ribbons, had big hair, brushed the hair, cut his hair, his long hair, shaving her hair, shave his beard, dress cut through, rolled up sleeves, hair huge and, cutti..."


# Generate story

In [63]:
VERSION = 'v1'
PROMPTS = {
    'v0': {
        'PROMPT_FIRST_PREFIX': 'Write the beginning paragraph of a story about',
        'PROMPT_NEXT_PREFIX': 'Write the next paragraph of the story, but now make it about',
        'PROMPT_SUFFIX': ' "{expl}". Make sure it contains several references to "{expl}".',
    },
    'v1': {
        'PROMPT_FIRST_PREFIX': 'Write the beginning paragraph of a story told in first person. The story should be about',
        'PROMPT_NEXT_PREFIX': 'Write the next paragraph of the story, but now make it about',
        'PROMPT_SUFFIX': ' "{expl}". Make sure it contains several references to "{expl}".',
    },
}
PROMPT_FIRST_PREFIX = PROMPTS[VERSION]['PROMPT_FIRST_PREFIX']
PROMPT_NEXT_PREFIX = PROMPTS[VERSION]['PROMPT_NEXT_PREFIX']
PROMPT_SUFFIX = PROMPTS[VERSION]['PROMPT_SUFFIX']

# get a list of prompts
expls = [v[0] for v in voxels]
def get_prompts_basic(expls):
    prompt_init = PROMPT_FIRST_PREFIX + PROMPT_SUFFIX
    prompt_continue = PROMPT_NEXT_PREFIX + PROMPT_SUFFIX
    prompts = [prompt_init.format(expl=expls[0])] + [prompt_continue.format(expl=expl) for expl in expls[1:]]
    return prompts
prompts = get_prompts_basic(expls)
for p in prompts:
    print(p)

Write the beginning paragraph of a story told in first person. The story should be about "relationships". Make sure it contains several references to "relationships".
Write the next paragraph of the story, but now make it about "time". Make sure it contains several references to "time".
Write the next paragraph of the story, but now make it about "looking or staring". Make sure it contains several references to "looking or staring".


In [71]:
# generate the paragraphs (these prefixes need get replace when the story is getting long)
paragraphs = mprompt.llm.get_paragraphs(prompts, PROMPT_FIRST_PREFIX=PROMPT_FIRST_PREFIX, PROMPT_NEXT_PREFIX=PROMPT_NEXT_PREFIX)
for para in paragraphs:
    print(para)

cached!
cached!
not cached
I've always been fascinated by relationships. Whether it's the bond between a parent and child, the love between two partners, or the friendship between two best friends, there's something magical about the way people connect with each other. As a child, I would watch my parents interact with each other and wonder how they managed to stay together for so long. As I grew older, I began to experience my own relationships and learned that they can be both beautiful and complicated. From heartbreak to true love, relationships have taught me some of life's most valuable lessons.
Time has always been a factor in my relationships. It's amazing how much can change over time. I've seen friendships fade away as people grow apart, and I've watched as couples drift apart after years of being together. But I've also seen relationships that have stood the test of time, where two people have grown together and become stronger because of it. Time can be both a blessing and a

# Test synthetic data contains concept

In [72]:
# mod = EmbDiffModule()
val = D5_Validator()

loading model weights


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

done


### Visualize single story

In [73]:
story_running = ''
scores = {}
for i in range(len(expls)):
# for i in range(1):
    expl = expls[i].lower()
    text = paragraphs[i]
    words = text.split()
    prompt = prompts[i]

    ngrams = imodelsx.util.generate_ngrams_list(text.lower(), ngrams=3)
    ngrams = [words[0], words[0] + ' ' + words[1]] + ngrams

    # # embdiff-based viz
    # mod._init_task(expl)    
    # neg_dists = mod(ngrams)
    # assert len(ngrams) == len(words) == len(neg_dists)
    # # neg_dists = scipy.special.softmax(neg_dists)
    # # plt.plot(neg_dists)
    # # plt.plot(moving_average(neg_dists, n=5))
    # neg_dists = moving_average(neg_dists, n=3)
    # neg_dists = (neg_dists - neg_dists.min()) / (neg_dists.max() - neg_dists.min())
    # neg_dists = neg_dists / 2 + 0.5 # shift to 0.5-1 range
    # s = mprompt.viz.colorize(words, neg_dists, title=expl, subtitle=prompt)

    # validator-based viz
    probs = np.array(val.validate_w_scores(expl, ngrams))
    probs_disp = mprompt.viz.moving_average(probs, n=3)
    probs_disp = probs_disp / 2 + 0.5 # shift to 0.5-1 range
    s = mprompt.viz.colorize(words, probs_disp, title=expl, subtitle=prompt)
    
    # viz
    display(HTML(s))
    story_running += ' ' + s

with open('../results/story_running.html', 'w') as f:
    f.write(story_running)

### Heatmap of story <> explanation matches

In [ ]:
n = len(expls)
scores = np.zeros((n, n))
for i in tqdm(range(n)):
    expl = expls[i]
    for j in range(n):
        text = paragraphs[j].lower()
        words = text.split()

        ngrams = imodelsx.util.generate_ngrams_list(text, ngrams=3)
        ngrams = [words[0], words[0] + ' ' + words[1]] + ngrams

        # validator-based viz
        probs = np.array(val.validate_w_scores(expl, ngrams)) > 0.5
        scores[i, j] = probs.mean()

In [ ]:
s = scores
# s = scipy.special.softmax(scores, axis=1)
# s = (s - s.min()) / (s.max() - s.min())
plt.style.use('dark_background')
plt.figure(figsize=(6, 5))
plt.imshow(s)
plt.xticks(range(n), expls, rotation=90)
plt.yticks(range(n), expls)
plt.ylabel('Explanation for generation')
plt.xlabel('Explanation for matching')
plt.colorbar(label='Fraction of matching ngrams')
plt.tight_layout()
plt.show()

# Test modules on the generated stories

In [ ]:
n = len(expls)
scores = np.zeros((n, n))
scores_max = np.zeros((n, n))
all_scores = {}
all_ngrams = {}
for i in tqdm(range(n)):
    # expl = expls[i]
    (expl, subject, voxel_num_best) = voxels[i]
    mod = fMRIModule(voxel_num_best=voxel_num_best, subject=subject)
    ngrams_list = []
    ngrams_scores_list = []
    for j in range(n):
        text = paragraphs[j].lower()
        words = text.split()
        ngrams_story = imodelsx.util.generate_ngrams_list(text, ngrams=3)
        ngrams_story = [words[0], words[0] + ' ' + words[1]] + ngrams_story
        ngrams_list.append(ngrams_story)

        # get mean score for each story
        ngrams_scores_story = mod(ngrams_story)
        ngrams_scores_list.append(ngrams_scores_story)
        scores[i, j] = ngrams_scores_story.mean()
        scores_max[i, j] = ngrams_scores_story.max()
    
    all_scores[i] = deepcopy(ngrams_scores_list)
    all_ngrams[i] = deepcopy(ngrams_list)

In [ ]:
s = scores
s = scipy.special.softmax(scores, axis=1)
# s = (s - s.min()) / (s.max() - s.min())
plt.style.use('dark_background')
plt.figure(figsize=(6, 5))
plt.imshow(s)
plt.xticks(range(n), expls, rotation=90)
plt.yticks(range(n), expls)
plt.ylabel('Explanation for generation')
plt.xlabel('Explanation of voxel used for evaluation')
plt.colorbar(label='Mean voxel response')
plt.tight_layout()
plt.show()

### Module responses for single story

In [ ]:
pd.set_option('display.max_rows', 120)
for i in range(1): #len(voxels)):
    row = rows_check.iloc[i]
    display(row[['subject', 'module_num', 'top_explanation_init_strs', 'explanation_init_ngrams', 'top_ngrams_module_correct']])
    mod = fMRIModule(voxel_num_best=row.module_num, subject=row.subject)
    # show all rows
    # display(
    #     pd.DataFrame.from_dict({
    #         'score': all_scores[i][i],
    #         'ngram': all_ngrams[i][i],
    #     }).sort_values('score', ascending=False).head(10)
    # )

In [ ]:
x = row['explanation_init_ngrams']

In [ ]:
out = mod(x, return_all=True)

In [ ]:
p = out[:, row.module_num]

In [ ]:
out